#Reading all Input Files

In [1]:
pip install pyxlsb

Note: you may need to restart the kernel to use updated packages.


In [3]:
from imap_tools import MailBox, AND
import imaplib, email
import os 
from datetime import datetime, timedelta 

EMAIL = os.getenv("EMAIL")
PASSWORD = os.getenv("EMAIL_APP_PASSWORD")
IMAP_SERVER = "imap.gmail.com"

print("Using Gmail account:", EMAIL)

#folder where attachments will be donwloaded or saved 
download_folder = "attachments" 

#create folder if doesnt exist, if it does then use the same folder! 
os.makedirs(download_folder, exist_ok = True) 
print(f"Attachments will be saved in: {download_folder}/")

#seaching for right email-
date_cutoff = (datetime.now() - timedelta(days=14)).date()

with MailBox(IMAP_SERVER).login(EMAIL,PASSWORD) as mailbox:
    # Step 1: Fetch emails only from client
    messages = list(mailbox.fetch(AND(from_="1.someother.5@gmail.com", date_gte = date_cutoff)))
    
    # Step 2: Filter in Python for subject containing our phrase (case-insensitive)
    filtered_msgs = [m for m in messages if "input file requirement" in m.subject.lower()]
    
    if not filtered_msgs:
        print("No matching emails found.")
    else:
        # Sort emails by date and pick the latest
        latest_msg = sorted(messages, key=lambda m: m.date, reverse=True)[0]
        
        print(f"Found latest email!")
        print(f"Subject: {latest_msg.subject}")
        print(f"From: {latest_msg.from_}")
        print(f"Date: {latest_msg.date}")

        #save attachments
        for att in latest_msg.attachments:
            filepath = os.path.join(download_folder, att.filename)
            with open(filepath, "wb") as f:
                f.write(att.payload)
            print(f"Downloaded: {att.filename}")

Using Gmail account: kneigapula@gmail.com
Attachments will be saved in: attachments/
Found latest email!
Subject: [Client] -> Input File Requirement_10-09-2025
From: 1.someother.5@gmail.com
Date: 2025-09-10 19:05:52+05:30
Downloaded: Store List Information.csv
Downloaded: Store Class Config.csv
Downloaded: Store Master.tsv
Downloaded: Store-Channel Information.csv
Downloaded: Markdown File_Updated.xlsx
Downloaded: MODEL MASTER - Effective 29 May 2025.xlsx
Downloaded: Ean Master.tsv
Downloaded: Style Master.tsv
Downloaded: Transfer Out_Updated.csv


In [4]:
import pandas as pd

markdown_df = pd.read_excel(r'C:\Users\Admin\Documents\Asics\attachments\Markdown File_Updated.xlsx', sheet_name = 'Model Master', usecols = ['Model no. ', 'Current Disc'],
                            na_values = '(blank)') 
adhar_df = pd.read_excel(r'C:\Users\Admin\Documents\Asics\attachments\MODEL MASTER - Effective 29 May 2025.xlsx', sheet_name = 'Model Master', usecols = ['Model no. ','SKU Cannot Be Allocated to Full-Priced Stores (marked "YES")',
                        'SKU Cannot Be Allocated to Factory Outlets (marked "YES")'], skiprows=1) 
outwards_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Transfer Out_Updated.csv') 
store_list_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Store List Information.csv')
store_channel_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Store-Channel Information.csv')
store_classconfig_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Store Class Config.csv')
style_master_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Style Master.tsv', sep='\t')
ean_master_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Ean Master.tsv', sep='\t')
store_master_df = pd.read_csv(r'C:\Users\Admin\Documents\Asics\attachments\Store Master.tsv', sep='\t')

In [5]:
style_master_formerge = style_master_df[['style_code','season']]
markdown_df = pd.merge(markdown_df, style_master_formerge, how = "left", left_on='Model no. ', right_on='style_code') 
markdown_df.drop(columns=['style_code'], inplace=True)
markdown_df.head(2) 

,Model no.,Current Disc,season
0,124627.165,0.0,NaN
1,127803.0934,0.0,NaN


In [6]:
markdown_seasonfilter = markdown_df[markdown_df['season'].isin(['SS23', 'SS24', 'SS25', 'AW23', 'AW24'])]
markdown_seasonfilter.head(2)

,Model no.,Current Disc,season
3946,1173A006.001,0.40625,AW23
4804,1191A214.100,0.30000,AW24


In [7]:
discountA = store_classconfig_df[store_classconfig_df['class']=='A']['discount'].iloc[0]
condition = markdown_seasonfilter['Current Disc'] > discountA
styles_for_A = markdown_seasonfilter[condition]
styles_for_A.head()

,Model no.,Current Disc,season
3946,1173A006.001,0.406250,AW23
5051,1173A005.101,0.714286,SS24
5052,1173A009.002,0.655172,SS24
5683,1202A024.100,0.500000,AW24
5965,1194A064.001,0.600000,SS24


In [8]:
discountB = store_classconfig_df[store_classconfig_df['class']=='B']['discount'].iloc[0]
condition = markdown_seasonfilter['Current Disc'] > discountB
styles_for_B = markdown_seasonfilter[condition]
styles_for_B.head() 

,Model no.,Current Disc,season
5051,1173A005.101,0.714286,SS24
5052,1173A009.002,0.655172,SS24
5965,1194A064.001,0.600000,SS24
7004,1194A076.103,0.600000,SS24
7251,1113A009.020,0.671141,SS23


In [9]:
stores_A = store_list_df[store_list_df['class']=='A']['store_code'] 
stores_B = store_list_df[store_list_df['class']=='B']['store_code']

#Preparation of Cross Merge and Merging the files

In [10]:
df_stores_A = pd.DataFrame({'store_code':stores_A, 'key':1})
df_stores_B = pd.DataFrame({'store_code':stores_B, 'key':1})

df_styles_A = pd.DataFrame({'style_code':styles_for_A['Model no. '], 'key':1})
df_styles_B = pd.DataFrame({'style_code':styles_for_B['Model no. '], 'key':1})

In [11]:
cross_merge_A = pd.merge(df_stores_A, df_styles_A, on="key").drop('key', axis=1)
cross_merge_B = pd.merge(df_stores_B, df_styles_B, on="key").drop('key', axis=1)

In [12]:
cross_merge_AB = pd.concat([cross_merge_A, cross_merge_B], ignore_index=True)

In [13]:
final_markdown = pd.merge(cross_merge_AB, store_channel_df, how="left", on="store_code") 
final_markdown['enabled'] = 1
final_markdown['description'] = "markdown logic"
final_markdown.head()

,store_code,style_code,Store Name,channel,enabled,description
0,VCT,1173A006.001,ASICS VIVO,EBO,1,markdown logic
1,VCT,1173A005.101,ASICS VIVO,EBO,1,markdown logic
2,VCT,1173A009.002,ASICS VIVO,EBO,1,markdown logic
3,VCT,1202A024.100,ASICS VIVO,EBO,1,markdown logic
4,VCT,1194A064.001,ASICS VIVO,EBO,1,markdown logic


#Final Markdown Logic File

In [14]:
final_markdown = final_markdown.drop(columns = ['Store Name'])
final_markdown.head()

,store_code,style_code,channel,enabled,description
0,VCT,1173A006.001,EBO,1,markdown logic
1,VCT,1173A005.101,EBO,1,markdown logic
2,VCT,1173A009.002,EBO,1,markdown logic
3,VCT,1202A024.100,EBO,1,markdown logic
4,VCT,1194A064.001,EBO,1,markdown logic


In [15]:
final_markdown.to_csv(r'C:\Users\Admin\Documents\Asics\attachments\markdown sample.tsv',sep='\t',index=False)

In [16]:
style_master_forfilter = style_master_df[['style_code']]
style_master_forfilter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22083 entries, 0 to 22082
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   style_code  22083 non-null  object
dtypes: object(1)
memory usage: 172.7+ KB


In [17]:
adhar_modif = pd.merge(adhar_df, style_master_forfilter, how="inner", left_on="Model no. ", right_on="style_code")
adhar_modif.head()

,Model no.,"SKU Cannot Be Allocated to Full-Priced Stores (marked ""YES"")","SKU Cannot Be Allocated to Factory Outlets (marked ""YES"")",style_code
0,1011A002.002,NaN,NaN,1011A002.002
1,1011A002.401,NaN,NaN,1011A002.401
2,1011A006.001,NaN,NaN,1011A006.001
3,1011A006.020,NaN,NaN,1011A006.020
4,1011A008.001,NaN,NaN,1011A008.001


In [18]:
adhar_modif_EBO = adhar_modif[adhar_modif['SKU Cannot Be Allocated to Full-Priced Stores (marked "YES")']=='YES']['Model no. ']
adhar_modif_EBO = pd.DataFrame({'style_code':adhar_modif_EBO, 'channel':'EBO'})
adhar_modif_EBO.head()

,style_code,channel
429,718W01.0692,EBO
431,718W01.8044,EBO
1281,J702N.600,EBO
3051,J702N.9001,EBO
5240,1103A024.005,EBO


In [19]:
adhar_modif_FO = adhar_modif[adhar_modif['SKU Cannot Be Allocated to Factory Outlets (marked "YES")']=='YES']['Model no. ']
adhar_modif_FO = pd.DataFrame({'style_code':adhar_modif_FO, 'channel':'FO'})
adhar_modif_FO.head()

,style_code,channel
429,718W01.0692,FO
431,718W01.8044,FO
1281,J702N.600,FO
3051,J702N.9001,FO
5240,1103A024.005,FO


In [20]:
import numpy as np 

adhar_final = pd.concat([adhar_modif_EBO, adhar_modif_FO], ignore_index=True)
adhar_final['enabled'] = 1
adhar_final['description'] = "adhar logic"
adhar_final['store_code'] = np.nan
adhar_final.head()

,style_code,channel,enabled,description,store_code
0,718W01.0692,EBO,1,adhar logic,NaN
1,718W01.8044,EBO,1,adhar logic,NaN
2,J702N.600,EBO,1,adhar logic,NaN
3,J702N.9001,EBO,1,adhar logic,NaN
4,1103A024.005,EBO,1,adhar logic,NaN


#Final Adhar logic file

In [21]:
adhar_final.to_csv(r'C:\Users\Admin\Documents\Asics\attachments\adhar sample.tsv', sep='\t',index=False)

In [22]:
sportstyle_styles = pd.DataFrame({
    'style_code':['3203A077.001', '3203A078.001', '3203A078.400', '3203A079.001', '3203A079.020', '3203A079.100', '3203A079.500', '3203A079.600', '3203A082.001', '3203A082.500', '3203A086.001', '3203A086.020', '3203A087.001', '3203A087.500'],
})
sportstyle_styles.head()
    

,style_code
0,3203A077.001
1,3203A078.001
2,3203A078.400
3,3203A079.001
4,3203A079.020


In [23]:
channels = pd.DataFrame({'channel': ['EBO','FO']})
channels.head()

,channel
0,EBO
1,FO


In [24]:
sportstyle_styles['key'] = 1
channels['key'] = 1

In [25]:
sportstyle_final = pd.merge(sportstyle_styles,channels, on="key").drop('key',axis=1)
sportstyle_final.head()

,style_code,channel
0,3203A077.001,EBO
1,3203A077.001,FO
2,3203A078.001,EBO
3,3203A078.001,FO
4,3203A078.400,EBO


In [26]:
sportstyle_final['enabled'] = 1
sportstyle_final['description'] = "sportstyle styles"
sportstyle_final['store_code'] = np.nan
sportstyle_final.head()

,style_code,channel,enabled,description,store_code
0,3203A077.001,EBO,1,sportstyle styles,NaN
1,3203A077.001,FO,1,sportstyle styles,NaN
2,3203A078.001,EBO,1,sportstyle styles,NaN
3,3203A078.001,FO,1,sportstyle styles,NaN
4,3203A078.400,EBO,1,sportstyle styles,NaN


#Sportstyle Logic Final File

In [27]:
from datetime import datetime
base_path = r'C:\Users\Admin\Documents\Asics\attachments'
file_prefix = 'sportstyle_sample'
full_output_path = f"{base_path}\\{file_prefix}_{datetime.now().strftime('%d_%m_%Y')}.tsv"
sportstyle_final.to_csv(full_output_path, sep='\t', index=False)

In [28]:
styles_in_masters = style_master_df['style_code']
styles_in_masters.head()

0    1011A002.002
1    1011A002.021
2    1011A002.022
3    1011A002.401
4    1011A006.001
Name: style_code, dtype: object

In [29]:
outwards_df = pd.merge(outwards_df, ean_master_df, how="inner", left_on = "SKU Code", right_on = "ean")
outwards_df.head()

,Channel,Origin Location,SKU Code,Quantity,Date,Destination Location (Warehouse Code),Document Number,ean,style,size,mrp
0,EBO,ABU,2011C353.001.2XL,1,07-04-2025,M08,ABU_20250407OUT_PW03_236,2011C353.001.2XL,2011C353.001,2XL,63.30
1,EBO,ABU,2011C353.001.L,1,07-04-2025,M08,ABU_20250407OUT_PW03_236,2011C353.001.L,2011C353.001,L,63.30
2,EBO,ABU,2011C353.001.M,1,07-04-2025,M08,ABU_20250407OUT_PW03_236,2011C353.001.M,2011C353.001,M,63.30
3,EBO,ABU,2011C353.001.XL,1,07-04-2025,M08,ABU_20250407OUT_PW03_236,2011C353.001.XL,2011C353.001,XL,63.30
4,EBO,ABU,2011C398.700.L,2,07-04-2025,M08,ABU_20250407OUT_PW03_236,2011C398.700.L,2011C398.700,L,59.63


In [30]:
outwards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27846 entries, 0 to 27845
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Channel                                27846 non-null  object 
 1   Origin Location                        27846 non-null  object 
 2   SKU Code                               27846 non-null  object 
 3   Quantity                               27846 non-null  int64  
 4   Date                                   27846 non-null  object 
 5   Destination Location (Warehouse Code)  27846 non-null  object 
 6   Document Number                        27846 non-null  object 
 7   ean                                    27846 non-null  object 
 8   style                                  27846 non-null  object 
 9   size                                   27846 non-null  object 
 10  mrp                                    27846 non-null  float64
dtypes:

In [31]:
outwards_df['Date'] = pd.to_datetime(outwards_df['Date'], format='%d-%m-%Y')
outwards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27846 entries, 0 to 27845
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Channel                                27846 non-null  object        
 1   Origin Location                        27846 non-null  object        
 2   SKU Code                               27846 non-null  object        
 3   Quantity                               27846 non-null  int64         
 4   Date                                   27846 non-null  datetime64[ns]
 5   Destination Location (Warehouse Code)  27846 non-null  object        
 6   Document Number                        27846 non-null  object        
 7   ean                                    27846 non-null  object        
 8   style                                  27846 non-null  object        
 9   size                                   27846 non-null  object

In [32]:
from datetime import timedelta
today_date = datetime.now()
date_benchmark = today_date - timedelta(weeks=6)

print(f"\nToday's date: {today_date.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Date benchmark (6 weeks ago): {date_benchmark.strftime('%Y-%m-%d %H:%M:%S')}")

condition = outwards_df['Date'] >= date_benchmark

outward_6weeksfilter = outwards_df[condition]
outward_6weeksfilter.head()


Today's date: 2025-09-15 19:53:15
Date benchmark (6 weeks ago): 2025-08-04 19:53:15


,Channel,Origin Location,SKU Code,Quantity,Date,Destination Location (Warehouse Code),Document Number,ean,style,size,mrp


In [33]:
store_codes = ['ABU', 'JAS', 'PAS', 'PSG', 'PWP', 'VCT', 'AWG', 'NEX', 'TAK', 'VLC', 'CCP', 'AIM']

outward_6weeksfilter['Origin Location'] = outward_6weeksfilter['Origin Location'].replace('PS', 'PSG')
outward_6weeksfilter['Origin Location'] = outward_6weeksfilter['Origin Location'].replace('VC', 'VCT')

In [34]:
condition = outward_6weeksfilter['Origin Location'].isin(store_codes) 
outward_6weeksfilter = outward_6weeksfilter[condition]
outward_6weeksfilter.head() 

,Channel,Origin Location,SKU Code,Quantity,Date,Destination Location (Warehouse Code),Document Number,ean,style,size,mrp


In [37]:
outwards_final = outward_6weeksfilter.pivot_table(index = ['Channel', 'Origin Location', 'style'], values='Quantity',aggfunc='sum') 
outwards_final.head()

,,
Channel,Origin Location,style


In [38]:
#outwards_final = outwards_final.drop(columns = ['Quantity'])
outwards_final.head()

,,
Channel,Origin Location,style


#Outwards Logic Final File:

In [39]:
outwards_final['enabled'] = 1
outwards_final['description'] = "outwards logic"

outwards_final.head()

,,,enabled,description
Channel,Origin Location,style,,


In [40]:
outwards_final = outwards_final.reset_index()
outwards_final.head()

,Channel,Origin Location,style,enabled,description


In [41]:
#outwards_final = outwards_final.reset_index() 
#outwards_final.drop(columns = ['level_0'], inplace=True)
outwards_final.head()

,Channel,Origin Location,style,enabled,description


In [42]:
outwards_final.rename(columns = {
    'Channel': 'channel',
    'Origin Location':'store_code',
    'style':'style_code'}, inplace=True)
outwards_final.drop(columns=['channel'], inplace=True)
outwards_final.head()

,store_code,style_code,enabled,description


In [43]:
outwards_final = pd.merge(outwards_final, store_channel_df, on="store_code")
outwards_final.head()

,store_code,style_code,enabled,description,Store Name,channel


In [44]:
outwards_final = outwards_final.drop(columns = ['Store Name'])
outwards_final.head()

,store_code,style_code,enabled,description,channel


In [45]:
o_file_prefix = 'outwards sample'
full_output_path = f"{base_path}\\{o_file_prefix}_{datetime.now().strftime('%d_%m_%Y')}.tsv"
outwards_final.to_csv(full_output_path, sep='\t', index=False) 

In [46]:
import pandas as pd 
from datetime import datetime, timedelta
lp_blanks_df = pd.read_excel(r'C:\Users\Admin\Documents\Asics\Copy of Copy of SS25 Launch Form (ASICS).xlsx', usecols = ['Item No.','ASICS PS','ASICS PARAGON','ASICS VIVO',
    'ASICS BUGIS','ASICS JEWEL','ASICS PARKWAY','ASICS NEX','ASICS VELOCITY','ASICS WESTGATE','ASICS TAKA','ASICS IMM','ASICS CCP'], skiprows=5)
lp_blanks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166 entries, 0 to 1165
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Item No.        1142 non-null   object 
 1   ASICS PS        774 non-null    float64
 2   ASICS PARAGON   849 non-null    float64
 3   ASICS VIVO      735 non-null    float64
 4   ASICS BUGIS     752 non-null    float64
 5   ASICS JEWEL     499 non-null    float64
 6   ASICS PARKWAY   446 non-null    object 
 7   ASICS NEX       441 non-null    object 
 8   ASICS VELOCITY  394 non-null    object 
 9   ASICS WESTGATE  428 non-null    object 
 10  ASICS TAKA      378 non-null    object 
 11  ASICS IMM       92 non-null     object 
 12  ASICS CCP       64 non-null     object 
dtypes: float64(5), object(8)
memory usage: 118.6+ KB


In [47]:
import numpy as np
lp_blanks_df = lp_blanks_df.replace(np.nan, 0)

store_columns = [col for col in lp_blanks_df.columns if col != 'Item No.']
for col in store_columns:
    lp_blanks_df[col] = pd.to_numeric(lp_blanks_df[col], errors='coerce')

lp_blanks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166 entries, 0 to 1165
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Item No.        1166 non-null   object 
 1   ASICS PS        1166 non-null   float64
 2   ASICS PARAGON   1166 non-null   float64
 3   ASICS VIVO      1166 non-null   float64
 4   ASICS BUGIS     1166 non-null   float64
 5   ASICS JEWEL     1166 non-null   float64
 6   ASICS PARKWAY   1144 non-null   float64
 7   ASICS NEX       1161 non-null   float64
 8   ASICS VELOCITY  1161 non-null   float64
 9   ASICS WESTGATE  1160 non-null   float64
 10  ASICS TAKA      1154 non-null   float64
 11  ASICS IMM       1163 non-null   float64
 12  ASICS CCP       1163 non-null   float64
dtypes: float64(12), object(1)
memory usage: 118.6+ KB


In [48]:
lp_blanks_df['Item No.'] = lp_blanks_df['Item No.'].astype(str)
indexes_to_drop = lp_blanks_df[lp_blanks_df['Item No.'] == '0'].index
lp_blanks_df = lp_blanks_df.drop(indexes_to_drop)
lp_blanks_df.head()

,Item No.,ASICS PS,ASICS PARAGON,ASICS VIVO,ASICS BUGIS,ASICS JEWEL,ASICS PARKWAY,ASICS NEX,ASICS VELOCITY,ASICS WESTGATE,ASICS TAKA,ASICS IMM,ASICS CCP
0,1011B974.002,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0,0.0
1,1011B975.002,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0,0.0
2,1011B974.500,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0,0.0
3,1011B974.400,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0,0.0
4,1012B765.002,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0,0.0


In [49]:
lp_blanks_pivot = lp_blanks_df.melt(id_vars=['Item No.'], var_name = 'Store Name', value_name='depth')
lp_blanks_pivot.head()

,Item No.,Store Name,depth
0,1011B974.002,ASICS PS,12.0
1,1011B975.002,ASICS PS,12.0
2,1011B974.500,ASICS PS,12.0
3,1011B974.400,ASICS PS,12.0
4,1012B765.002,ASICS PS,12.0


In [50]:
lp_blanks = lp_blanks_pivot[lp_blanks_pivot['depth'] == 0]
lp_blanks.head() 

,Item No.,Store Name,depth
8,1101A070.600,ASICS PS,0.0
9,1011B888.800,ASICS PS,0.0
10,1041A409.103,ASICS PS,0.0
11,1012B689.800,ASICS PS,0.0
12,1042A236.102,ASICS PS,0.0


In [51]:
lp_blanks = pd.merge(lp_blanks, store_channel_df, on = "Store Name")
lp_blanks.head()

,Item No.,Store Name,depth,store_code,channel
0,1101A070.600,ASICS PS,0.0,PSG,EBO
1,1011B888.800,ASICS PS,0.0,PSG,EBO
2,1041A409.103,ASICS PS,0.0,PSG,EBO
3,1012B689.800,ASICS PS,0.0,PSG,EBO
4,1042A236.102,ASICS PS,0.0,PSG,EBO


In [52]:
lp_blanks = lp_blanks.drop(columns = ['depth', 'Store Name'])
lp_blanks.rename(columns = {
    'Item No.': 'style_code'}, inplace=True) 
lp_blanks.head()

,style_code,store_code,channel
0,1101A070.600,PSG,EBO
1,1011B888.800,PSG,EBO
2,1041A409.103,PSG,EBO
3,1012B689.800,PSG,EBO
4,1042A236.102,PSG,EBO


In [53]:
lp_blanks_final = pd.merge(lp_blanks, final_markdown, how="left", on = ['store_code', 'channel', 'style_code'],indicator=True)
lp_blanks_final['_merge'].unique()

['left_only']
Categories (3, object): ['left_only', 'right_only', 'both']

In [54]:
lp_blanks_final = lp_blanks_final[lp_blanks_final['_merge'] == 'left_only']
lp_blanks_final.head()

,style_code,store_code,channel,enabled,description,_merge
0,1101A070.600,PSG,EBO,NaN,NaN,left_only
1,1011B888.800,PSG,EBO,NaN,NaN,left_only
2,1041A409.103,PSG,EBO,NaN,NaN,left_only
3,1012B689.800,PSG,EBO,NaN,NaN,left_only
4,1042A236.102,PSG,EBO,NaN,NaN,left_only


In [55]:
lp_blanks_final['enabled'] = 1
lp_blanks_final['description'] = "LP blanks logic"
lp_blanks_final.drop(columns=['_merge'],inplace=True)
lp_blanks_final.head()

,style_code,store_code,channel,enabled,description
0,1101A070.600,PSG,EBO,1,LP blanks logic
1,1011B888.800,PSG,EBO,1,LP blanks logic
2,1041A409.103,PSG,EBO,1,LP blanks logic
3,1012B689.800,PSG,EBO,1,LP blanks logic
4,1042A236.102,PSG,EBO,1,LP blanks logic


#Launch Plan Blanks Final File

In [56]:
lp_file_prefix = 'lp blank sample'
full_output_path = f"{base_path}\\{lp_file_prefix}_{datetime.now().strftime('%d_%m_%Y')}.tsv"
lp_blanks_final.to_csv(full_output_path, sep='\t', index=False)

#FINAL MERGE ALL FILES

In [57]:
target_column = ['store_code', 'style_code', 'channel', 'enabled', 'description']
all_df = [lp_blanks_final, outwards_final, final_markdown, adhar_final, sportstyle_final] 

dfs_to_concat = []

for current_df in all_df:
    prepared_df = current_df[target_column]
    dfs_to_concat.append(prepared_df)

Merged_Exclusion = pd.concat(dfs_to_concat, ignore_index=True)

Merged_Exclusion.head()

,store_code,style_code,channel,enabled,description
0,PSG,1101A070.600,EBO,1,LP blanks logic
1,PSG,1011B888.800,EBO,1,LP blanks logic
2,PSG,1041A409.103,EBO,1,LP blanks logic
3,PSG,1012B689.800,EBO,1,LP blanks logic
4,PSG,1042A236.102,EBO,1,LP blanks logic


In [58]:
merged_file_prefix = 'Merged Exclusion List'
full_output_path = f"{base_path}\\{merged_file_prefix}_{datetime.now().strftime('%d_%m_%Y')}.tsv"
Merged_Exclusion.to_csv(full_output_path, sep='\t', index=False)

In [59]:
import os
import smtplib
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders

def pick_latest_by_mtime(folder):
    files = [f for f in os.listdir(folder) if f.startswith("Merged Exclusion List")]
    if not files:
        raise FileNotFoundError("No merged files found in " + folder)
    full_paths = [os.path.join(folder, f) for f in files]
    latest_path = max(full_paths, key=os.path.getmtime)
    return os.path.basename(latest_path)

# use environment variables (safer than hard-coding)
SENDER_EMAIL = os.getenv("EMAIL")
PASSWORD = os.getenv("EMAIL_APP_PASSWORD")
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

output_folder = "attachments"
latest_file = pick_latest_by_mtime(output_folder)
file_path = os.path.join(output_folder, latest_file)

# receiver_email: if you still have latest_msg from your IMAP fetch, use latest_msg.from_
receiver_email = latest_msg.from_ if 'latest_msg' in globals() else "1.someother.5@gmail.com"

# Build message
msg = MIMEMultipart()
msg["From"] = SENDER_EMAIL
msg["To"] = receiver_email
msg["Subject"] = f"[SPOC] -> Latest Automated Exclusion List - {latest_file}"
msg.attach(MIMEText("Hi,\n\nPlease find attached the latest exclusion list. \n\nFeel Free to reach out to us in case of any queries.\n\nThanks & Regards,\n[SPOC] ETL Team", "plain"))

# Attach file
with open(file_path, "rb") as attachment:
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f'attachment; filename="{latest_file}"')
    msg.attach(part)

# Send
with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
    server.starttls()
    server.login(SENDER_EMAIL, PASSWORD)
    server.send_message(msg)

print(f"Sent {latest_file} to {receiver_email}")


Sent Merged Exclusion List_15_09_2025.tsv to 1.someother.5@gmail.com
